In [1]:

import pandas as pd
import geopandas as gpd

In [13]:
# Load the bixi stations data
bixi_stations = gpd.read_file('../data/curated/bixi-stations.json')

# Filter stations that are within the city area
bixi_stations = bixi_stations[bixi_stations['WITHIN_CITY_AREA']]

# Display the first few rows
print(bixi_stations.head())

                               STATIONNAME  STATIONLATITUDE  STATIONLONGITUDE  \
0  Métro Champ-de-Mars (Viger / Sanguinet)        45.510253        -73.556777   
1  Métro Place-d'Armes (Viger / St-Urbain)        45.506314        -73.559671   
2               Émile-Duployé / Sherbrooke        45.527195        -73.564526   
3                       Marmier / St-Denis        45.531027        -73.598623   
4              du Parc-Lafontaine / Rachel        45.525512        -73.574245   

   WITHIN_CITY_AREA                    geometry  
0              True  POINT (-73.55678 45.51025)  
1              True  POINT (-73.55967 45.50631)  
2              True   POINT (-73.56453 45.5272)  
3              True  POINT (-73.59862 45.53103)  
4              True  POINT (-73.57424 45.52551)  


In [3]:
# Load the bixi trips data of 2024
bixi_trips = pd.read_csv('../data/raw/bixi/bixi-2024.csv')

In [4]:
# Filter bixi data to only include stations that are within the city area
bixi_trips = bixi_trips[bixi_trips['STARTSTATIONNAME'].isin(bixi_stations['STATIONNAME'])]
bixi_trips = bixi_trips[bixi_trips['ENDSTATIONNAME'].isin(bixi_stations['STATIONNAME'])]

In [5]:
# Display the first few rows
print(bixi_trips.head())

                          STARTSTATIONNAME   STARTSTATIONARRONDISSEMENT  \
0  Métro Champ-de-Mars (Viger / Sanguinet)                  Ville-Marie   
1  Métro Place-d'Armes (Viger / St-Urbain)                  Ville-Marie   
2               Émile-Duployé / Sherbrooke        Le Plateau-Mont-Royal   
3                       Marmier / St-Denis  Rosemont - La Petite-Patrie   
4                       Marmier / St-Denis  Rosemont - La Petite-Patrie   

   STARTSTATIONLATITUDE  STARTSTATIONLONGITUDE  \
0             45.510253             -73.556777   
1             45.506314             -73.559671   
2             45.527195             -73.564526   
3             45.531027             -73.598623   
4             45.531027             -73.598623   

                         ENDSTATIONNAME     ENDSTATIONARRONDISSEMENT  \
0       St-Antoine / Jean-Paul Riopelle                  Ville-Marie   
1         de Maisonneuve / Aylmer (est)                  Ville-Marie   
2  Parc Molson (d'Iberville / Be

In [8]:
# Transform STARTTIMEMS and ENDTIMEMS to time of day rounded down to hours and minutes
bixi_trips['STARTTIME'] = pd.to_datetime(bixi_trips['STARTTIMEMS'], unit='ms').dt.floor('min').dt.time
bixi_trips['ENDTIME'] = pd.to_datetime(bixi_trips['ENDTIMEMS'], unit='ms').dt.floor('min').dt.time

# Display the first few rows
print(bixi_trips[['STARTTIME', 'ENDTIME']].head())

  STARTTIME   ENDTIME
0  21:25:00  21:31:00
1  22:01:00  22:08:00
2  23:29:00  23:49:00
3  21:58:00  22:08:00
4  01:03:00  01:14:00


In [9]:
# Round it down to a 10 minute interval
bixi_trips['STARTTIMEINTERVAL'] = bixi_trips['STARTTIME'].apply(lambda x: x.replace(minute=x.minute//10*10))
bixi_trips['ENDTIMEINTERVAL'] = bixi_trips['ENDTIME'].apply(lambda x: x.replace(minute=x.minute//10*10))

# Display the first few rows
print(bixi_trips[['STARTTIMEINTERVAL', 'ENDTIMEINTERVAL']].head())

  STARTTIMEINTERVAL ENDTIMEINTERVAL
0          21:20:00        21:30:00
1          22:00:00        22:00:00
2          23:20:00        23:40:00
3          21:50:00        22:00:00
4          01:00:00        01:10:00


In [11]:
# group by start statation and start time interval and count the number of trips
bixi_starttrips_grouped = bixi_trips.groupby(['STARTSTATIONNAME', 'STARTTIMEINTERVAL']).size().reset_index(name='STARTCOUNT')

# group by end statation and end time interval and count the number of trips
bixi_endtrips_grouped = bixi_trips.groupby(['ENDSTATIONNAME', 'ENDTIMEINTERVAL']).size().reset_index(name='ENDCOUNT')

# Merge the two dataframes
bixi_trips_grouped = bixi_starttrips_grouped.merge(bixi_endtrips_grouped, left_on=['STARTSTATIONNAME', 'STARTTIMEINTERVAL'], right_on=['ENDSTATIONNAME', 'ENDTIMEINTERVAL'])

# Only keep 1 of the station name columns and time interval columns and keep the count columns
bixi_trips_grouped = bixi_trips_grouped[['STARTSTATIONNAME', 'STARTTIMEINTERVAL', 'STARTCOUNT', 'ENDCOUNT']]
bixi_trips_grouped.columns = ['STATIONNAME', 'TIMEINTERVAL', 'STARTCOUNT', 'ENDCOUNT']

# Display the first few rows
print(bixi_trips_grouped.head())

           STATIONNAME TIMEINTERVAL  STARTCOUNT  ENDCOUNT
0  10e avenue / Masson     00:00:00         259       289
1  10e avenue / Masson     00:10:00         255       257
2  10e avenue / Masson     00:20:00         218       231
3  10e avenue / Masson     00:30:00         241       223
4  10e avenue / Masson     00:40:00         213       210


In [14]:
# Merge the bixi stations data with the bixi trips data
bixi_stations_interval_counts = bixi_stations.merge(bixi_trips_grouped, on='STATIONNAME', how='left')

# Display the first few rows
print(bixi_stations_interval_counts.head())

                               STATIONNAME  STATIONLATITUDE  STATIONLONGITUDE  \
0  Métro Champ-de-Mars (Viger / Sanguinet)        45.510253        -73.556777   
1  Métro Champ-de-Mars (Viger / Sanguinet)        45.510253        -73.556777   
2  Métro Champ-de-Mars (Viger / Sanguinet)        45.510253        -73.556777   
3  Métro Champ-de-Mars (Viger / Sanguinet)        45.510253        -73.556777   
4  Métro Champ-de-Mars (Viger / Sanguinet)        45.510253        -73.556777   

   WITHIN_CITY_AREA                    geometry TIMEINTERVAL  STARTCOUNT  \
0              True  POINT (-73.55678 45.51025)     00:00:00         6.0   
1              True  POINT (-73.55678 45.51025)     00:10:00        12.0   
2              True  POINT (-73.55678 45.51025)     00:20:00         3.0   
3              True  POINT (-73.55678 45.51025)     00:30:00         4.0   
4              True  POINT (-73.55678 45.51025)     00:40:00         4.0   

   ENDCOUNT  
0       5.0  
1       4.0  
2       9.0  
